In this notebook we'll take a look at doing a sensitivity analysis by "functionizing" our models.  

In [1]:
import pandas as pd

# Read in the data first
profit = pd.read_excel('pizza-lp-munged.xlsx', sheet_name='profit', index_col=0)
resources = pd.read_excel('pizza-lp-munged.xlsx', sheet_name='resources', index_col=0)
rhs = pd.read_excel('pizza-lp-munged.xlsx', sheet_name='rhs', index_col=0)

# Pizza LP model as a function

In [2]:
import numpy as np
import pyomo.environ as pe

def convert_1d_array_to_dict(arr):
    '''Converts a 1-d ndarray, DataFrame, or Series to a paramater dictionary.'''
    if isinstance(arr, pd.DataFrame) or isinstance(arr, pd.Series):
        arr = arr.values
    arr = arr.squeeze()
    if np.shape(arr) == ():
        arr = np.array([arr])
    return {(i+1): arr[i] for i in range(len(arr))}


def convert_2d_array_to_dict(arr):
    '''Converts a 2-d ndarray, DataFrame or Series to a parameter dictionary.'''
    if isinstance(arr, pd.DataFrame) or isinstance(arr, pd.Series):
        arr = arr.values
    return {(i+1,j+1): arr[i,j] for i in range(arr.shape[0]) for j in range(arr.shape[1])}

# No more concrete model!
model = pe.AbstractModel()

# Decision variables are defined by an index object rather than a list.
model.pizza = pe.RangeSet(1, resources.shape[1])    # decisions are four pizza types
model.resource = pe.RangeSet(1, resources.shape[0]) # there are 5 resources

# Decision variables definition
model.x = pe.Var(model.pizza, domain=pe.NonNegativeReals)

# To enable the model to be "abstract", we use a Param object to store the data.
# NOTICE WE SET mutable=True, meaning we can change these values if we want to.
profit_dict = convert_1d_array_to_dict(profit)
model.profit_coef = pe.Param(model.pizza, mutable=True, initialize=profit_dict)

resource_dict = convert_2d_array_to_dict(resources)
model.resource_coef = pe.Param(model.resource, model.pizza, initialize=resource_dict)

rhs_dict = convert_1d_array_to_dict(rhs)
model.resource_rhs = pe.Param(model.resource, initialize=rhs_dict)

def objective_rule(model):
    return sum(model.profit_coef[p] * model.x[p] for p in model.pizza)
model.obj = pe.Objective(rule=objective_rule, sense=-1)

def resource_rule(model, r):
    return sum(model.resource_coef[r, p] * model.x[p] for p in model.pizza) <= model.resource_rhs[r]
model.resource_cons = pe.Constraint(model.resource, rule=resource_rule)


We'll need this helper function to see how the decisions change.

In [7]:
import pyomo

def parse_solution(model, instance, index=None):

    # Extract variable names
    var = [x for x in instance.component_map().keys() 
           if isinstance(getattr(instance, x), pyomo.core.base.var.IndexedVar)]
    
    # Create dataframe, distinguising between first and second stage variables
    dfs = []
    for v in var:
        df = pd.DataFrame([x.value for x in instance.component_map()[v].values()], columns=[v])
        dfs.append(df)
    dfs = pd.concat(dfs, axis=1)
    
    if index is not None:
        dfs.index = index
        
    return dfs

In [11]:
# Let's run a simple sensitivity analysis by changing plain pizza profit and re-solving
# for each value.
instance = model.create_instance()
opt = pe.SolverFactory('glpk')
for plain_profit in [8., 9., 10., 11., 12.]:
    
    instance.profit_coef[1] = plain_profit
    result = opt.solve(instance, tee=False) 
    
    print('plain profit coefficient = {}'.format(plain_profit))
    print('realized profit = {}'.format(instance.obj.expr()))
    print(parse_solution(model, instance, index=profit.index))
    print()

plain profit coefficient = 8.0
realized profit = 400.0
            x
pizza        
plain     0.0
meat     10.0
veggie    0.0
supreme  20.0

plain profit coefficient = 9.0
realized profit = 400.0
            x
pizza        
plain     0.0
meat     10.0
veggie    0.0
supreme  20.0

plain profit coefficient = 10.0
realized profit = 400.0
            x
pizza        
plain    10.0
meat      0.0
veggie    0.0
supreme  20.0

plain profit coefficient = 11.0
realized profit = 410.0
            x
pizza        
plain    10.0
meat      0.0
veggie    0.0
supreme  20.0

plain profit coefficient = 12.0
realized profit = 420.0
            x
pizza        
plain    10.0
meat      0.0
veggie    0.0
supreme  20.0

